In [4]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.preprocessing import scale
from statsmodels.base.model import GenericLikelihoodModel
%matplotlib inline

In [8]:
def vary(values, frac=0.1, round_dec=2):
    dev = frac * np.random.uniform(-1, 1, len(values))
    new_vals = np.array(values) * (1 + dev)
    new_vals = np.round_(new_vals, round_dec)
    return new_vals

In [10]:
#make trainDB
ones = np.ones(20)
meas1 = vary(ones)
meas4 = vary(1.5 * ones)
meas2 = vary(2 * ones)
meas5 = vary(2.5 * ones)
meas3 = vary(3 * ones)

trainXY = pd.DataFrame({"a" : meas1, 
                        "b" : meas4,
                        "c" : meas2,
                        "d" : meas5,
                        "e" : meas3
                        })
trainXY

,a,b,c,d,e
0,0.92,1.63,2.01,2.54,2.99
1,0.93,1.55,1.81,2.44,3.29
2,1.09,1.65,1.90,2.29,3.07
3,0.96,1.41,1.95,2.63,3.20
4,1.07,1.49,1.97,2.33,2.97
5,0.99,1.42,2.12,2.51,3.26
6,1.03,1.37,1.99,2.44,2.86
7,1.06,1.55,1.80,2.56,2.88
8,1.03,1.49,2.09,2.41,3.18
9,0.94,1.59,2.09,2.73,3.22


In [11]:
#make test cases
test1 = [1, 1.5, 2, 2.5, 3] # best case
test2 = [1, 0, 0, 0, 0] # 1 meas only, worst-ish case 
test3 = [1, 1, 1, 1, 1] # bad measurements
test4 = [0, 1.5, 2.5, 0, 0] # 2 meas only
test5 = [1, 0, 2, 0, 3]  # 3 meas only

In [12]:
def ll_calc(y_sim, y_mes, std):
    ll = np.sum(stats.norm.logpdf(y_sim, loc=y_mes, scale=std))
    return ll

def unc_calc(y_sim, y_mes, sim_unc_sq, mes_unc_sq):
    unc = ((y_sim - y_mes) / sim_unc_sq)**2 * (sim_unc_sq + mes_unc_sq)
    unc.replace([np.inf, -np.inf], 0, inplace=True)
    unc.fillna(0, inplace = True)
    return np.sqrt(unc.sum(axis=1))

In [16]:
test_sample = test1
row_unc = 0.1
test_unc = 0.1
X = trainXY # make a copy
trainXY['LogLikelihood'] = X.apply(lambda row: ll_calc(row, test_sample, row_unc), axis=1)
#############################################################################
# pandas converts each row to a series, so using .iloc[0] on the single-row #
# dataframe that is test_sample allows the function to work properly        #
#############################################################################

ValueError: ('operands could not be broadcast together with shapes (6,) (5,) ', 'occurred at index 0')

In [15]:
trainXY['LLUncertainty'] = X.apply(lambda row: unc_calc(row, test_sample, (row_unc*row)**2, (test_unc*test_sample)**2), axis=1)

NameError: ("name 'test_sample' is not defined", 'occurred at index 0')

In [37]:
trainXY.head(10)

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ba136,ba138,cs133,cs134,cs135,...,eu154,pu239,pu240,pu241,pu242,sm149,sm150,sm152,LogLikelihood,LLUncertainty
13925,6233.69,3960.268866,3.690,ce14x14,pwr,0.006953,1.8420,1.8310,0.001072,0.46170,...,0.002772,11.2400,1.38700,0.293200,0.028470,0.017760,0.31650,0.15900,-5671.018242,3.146719e+01
29766,5249.81,1909.887081,3.740,vver1000,pwr,0.005415,1.5530,1.5470,0.005026,0.39340,...,0.003054,10.8800,1.17300,0.274000,0.016660,0.017560,0.26080,0.13110,-5061.212201,3.134482e+01
21539,5249.81,5847.824653,2.990,vver1000,pwr,0.005036,1.5600,1.5520,0.000124,0.37430,...,0.001183,9.8970,1.09100,0.136500,0.014240,0.016620,0.25980,0.13260,-3735.277637,8.653219e+02
15032,8197.67,1.148565,3.690,ce14x14,pwr,0.009899,2.4120,2.3090,0.070030,0.60750,...,0.011600,12.9000,1.99500,0.921000,0.071750,0.013410,0.43140,0.21460,-8984.027644,3.954245e+01
37651,4142.81,708.508195,3.090,agr,agr,0.003838,1.2480,1.2420,0.007143,0.55390,...,0.001916,5.3080,0.69230,0.120400,0.006294,0.011060,0.20200,0.09891,-205.450192,2.671741e+01
72448,9494.18,2947.195294,0.711,candu37,phwr,0.008988,2.8780,2.8340,0.002791,0.75470,...,0.004441,6.8590,1.31000,0.160900,0.023540,0.009829,0.47330,0.27610,-1559.837986,3.673019e+01
50957,9091.96,3326.024034,3.630,agr,agr,0.015580,2.7030,2.6720,0.003021,1.22900,...,0.005257,8.6070,2.14600,0.583600,0.107100,0.011640,0.47320,0.24240,-3138.304335,3.896349e+01
40991,7636.02,4004.873486,3.090,agr,agr,0.010950,2.2820,2.2620,0.001109,1.01500,...,0.003043,7.5270,1.68600,0.334600,0.054830,0.011070,0.39030,0.20210,-1763.792917,3.439612e+01
26507,1524.67,4486.575797,3.740,vver1000,pwr,0.000749,0.4560,0.4564,0.000033,0.11430,...,0.000140,4.1110,0.14260,0.007331,0.000179,0.016110,0.06121,0.02874,-236.911532,1.329068e+04
52046,981.44,4410.359403,0.711,candu19,phwr,0.000305,0.3015,0.2986,0.000005,0.08673,...,0.000034,0.9459,0.01402,0.000128,0.000002,0.010650,0.03797,0.01770,-1204.804732,4.318022e+06


### Max LL: Reactor-dependent (paper presents it this way)

#### PWR

In [38]:
max_pwr = trainXY['LogLikelihood'].loc[trainXY['ReactorType'] == 'pwr'].max()
idx_pwr = trainXY['LogLikelihood'].loc[trainXY['ReactorType'] == 'pwr'].idxmax()
unc_pwr = float(trainXY['LLUncertainty'].loc[trainXY.index == idx_pwr])
print(f'Max Log Likelihood for PWRs: {max_pwr} +/- {unc_pwr}')
trainXY.loc[trainXY.index == idx_pwr, ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']]

Max Log Likelihood for PWRs: -172.46966586074987 +/- 228.7371919493878


,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
1013,pwr,99.56,3.1,1854.07,ce14x14


#### AGR

In [39]:
max_agr = trainXY['LogLikelihood'].loc[trainXY['ReactorType'] == 'agr'].max()
idx_agr = trainXY['LogLikelihood'].loc[trainXY['ReactorType'] == 'agr'].idxmax()
unc_agr = float(trainXY['LLUncertainty'].loc[trainXY.index == idx_agr])
print(f'Max Log Likelihood for AGRs: {max_agr} +/- {unc_agr}')
trainXY.loc[trainXY.index == idx_agr, ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']]

Max Log Likelihood for AGRs: -51.116212188388694 +/- 34.546105753176946


,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
36077,agr,1.135869,3.09,3271.84,agr


#### PHWR

In [40]:
max_phwr = trainXY['LogLikelihood'].loc[trainXY['ReactorType'] == 'phwr'].max()
idx_phwr = trainXY['LogLikelihood'].loc[trainXY['ReactorType'] == 'phwr'].idxmax()
unc_phwr = float(trainXY['LLUncertainty'].loc[trainXY.index == idx_phwr])
print(f'Max Log Likelihood for PHWRs: {max_phwr} +/- {unc_phwr}')
trainXY.loc[trainXY.index == idx_phwr, ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']]

Max Log Likelihood for PHWRs: 26.104720503166472 +/- 0.08383287423070052


,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
55865,phwr,2327.360723,0.711,4594.77,candu19
